In [1]:
! pip list | grep pyspark

pyspark                  2.4.4      


In [2]:
! pip install pyspark

In [0]:
from pyspark import *

In [0]:
sc = SparkContext(master="local", appName="test")

In [5]:
sc

<SparkContext master=local appName=test>

In [0]:
import urllib.request
response = urllib.request.urlopen('http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html')

In [0]:
import urllib
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')
data_file = './data_10_percent.gz'

lines = sc.textFile(data_file)

In [8]:
lines

./data_10_percent.gz MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
lines.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [11]:
print(sqlContext)

In [12]:
lines.collect()

IllegalArgumentException: ignored

In [0]:
fonction_split = lambda x : x.split(',')

In [0]:
data_parsees = lines.map(fonction_split)

In [15]:
data_parsees.take(3)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '235',
  '1337',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '29',
  '29',
  '1.00',
  '0.00',


In [0]:
from pyspark.sql.types import StructType

In [0]:
schema = types.StructType([
    types.StructField("duration", types.IntegerType(), True),
    types.StructField("protocol_type", types.StringType(), True),
    types.StructField("service", types.StringType(), True),
    types.StructField("flag", types.StringType(), True),
    types.StructField("src_bytes", types.IntegerType(), True),
    types.StructField("dst_bytes", types.IntegerType(), True),
    types.StructField("interactions", types.StringType(), True),
    ])

In [0]:
#from pyspark.sql.types import StructType
from pyspark.sql.types import *
schema = StructType([
    StructField("duration", IntegerType(), True),
    StructField("protocol_type", StringType(), True),
    StructField("service", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("src_bytes",IntegerType(), True),
    StructField("dst_bytes",IntegerType(), True),
    StructField("interactions",StringType(), True),
    ])

In [18]:
schema

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [19]:
newData = data_parsees.map(lambda key:(int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))
newData.take(3)

[(0, 'tcp', 'http', 'SF', 181, 5450, 'normal.'),
 (0, 'tcp', 'http', 'SF', 239, 486, 'normal.'),
 (0, 'tcp', 'http', 'SF', 235, 1337, 'normal.')]

In [20]:
dataFrame = sqlContext.createDataFrame(newData, schema)
dataFrame

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [0]:
dataFrame.show(10)


In [0]:
dataFrame.createTempView('interactions')

In [0]:
# Une tempview est une photo (tuples) du résultat d'une requete sql executée sur la BDD

In [0]:
filtre1 = dataFrame.filter(dataFrame.protocol_type=='tcp')

In [0]:
filtre2 = dataFrame.filter(filtre1.duration >1000)

In [0]:
filtre2 = filtre2.where("duration >1000")

In [26]:
filtre2.show()

+--------+-------------+--------+----+---------+---------+------------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+--------+----+---------+---------+------------+
|   12454|          tcp|  telnet|  SF|      432|    21181|     normal.|
|   10774|          tcp|  telnet|  SF|       75|      919|     normal.|
|    1023|          udp|   ntp_u|  SF|       48|       48|     normal.|
|    6087|          udp|domain_u|  SF|       46|       33|     normal.|
|   13368|          tcp|  telnet|  SF|     2987|    58843|     normal.|
|    3029|          udp|domain_u|  SF|       17|       17|     normal.|
|    2096|          udp|domain_u|  SF|       17|       46|     normal.|
|    3030|          udp|domain_u|  SF|       17|       17|     normal.|
|    3030|          udp|domain_u|  SF|       17|       17|     normal.|
|    3030|          udp|domain_u|  SF|       17|       17|     normal.|
|    3030|          udp|domain_u|  SF|       17|       17|     n

In [0]:
filtre3 = filtre2.filter(filtre2.dst_bytes == 0)

In [28]:
data = filtre3.show()

+--------+-------------+--------+----+---------+---------+------------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+--------+----+---------+---------+------------+
|    5057|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5059|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5051|          tcp|ftp_data|  SF|  5135678|        0|warezclient.|
|    5056|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5051|          tcp|ftp_data|  SF|  5135678|        0|warezclient.|
|    5039|          tcp|ftp_data|  SF|  5131424|        0|warezclient.|
|    5062|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5041|          tcp|   other|  SF|  5135678|        0|warezclient.|
|    5056|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5064|          tcp|ftp_data|  SF|  5133876|        0|warezclient.|
|    5043|          tcp|ftp_data|  SF|  5135678|        0|warezc

In [29]:
type(filtre3)


pyspark.sql.dataframe.DataFrame

In [30]:
filtre3.count()

IllegalArgumentException: ignored

In [0]:
! sudo apt install openjdk-8-jdk

In [0]:
filtre3.count()

In [0]:
! sudo update-alternatives --config javac

In [0]:
filtre3.count()

In [31]:
! java -version

openjdk version "11.0.5" 2019-10-15
OpenJDK Runtime Environment (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04)
OpenJDK 64-Bit Server VM (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04, mixed mode)


In [39]:
filtre3

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [42]:
filtre4 = lambda filtre3.duration : filtre3.duration if filtre3.duration>1000 else return None

SyntaxError: ignored